In [295]:
using CSV
using DataFrames
using XLSX
using Statistics
using Distributions
using Random
using Tables

In [211]:
phylum = CSV.read("../processed-data/otu-yield-per-plant.csv", DataFrame)
@info("Finish loading the data...")

┌ Info: Finish loading the data...
└ @ Main In[211]:2


In [212]:
otu = phylum[:,2:60]
otu = otu .* 100
phylum[:,2:60] .= otu
@info("Finish multiplying the data...")

┌ Info: Finish multiplying the data...
└ @ Main In[212]:4


In [213]:
# Split the data into varieties 
var_idx = size(phylum)[2] -1

red_low = filter(row -> (row.Variety2 == "Red" && row.Yield_per_plant == 0), phylum)
red_high = filter(row -> (row.Variety2 == "Red" && row.Yield_per_plant == 1), phylum)

white_low = filter(row -> (row.Variety2 == "White" && row.Yield_per_plant == 0), phylum)
white_high = filter(row -> (row.Variety2 == "White" && row.Yield_per_plant == 1), phylum)

yellow_low = filter(row -> (row.Variety2 == "Yellow" && row.Yield_per_plant == 0), phylum)
yellow_high = filter(row -> (row.Variety2 == "Yellow" && row.Yield_per_plant == 1), phylum)

russet_low = filter(row -> (row.Variety2 == "Russet" && row.Yield_per_plant == 0), phylum)
russet_high = filter(row -> (row.Variety2 == "Russet" && row.Yield_per_plant == 1), phylum)

red_little_low = filter(row -> (row.Variety2 == "RedLittle" && row.Yield_per_plant == 0), phylum)
red_little_high = filter(row -> (row.Variety2 == "RedLittle" && row.Yield_per_plant == 1), phylum)

yellow_little_low = filter(row -> (row.Variety2 == "YellowLittle" && row.Yield_per_plant == 0), phylum)
yellow_little_high = filter(row -> (row.Variety2 == "YellowLittle" && row.Yield_per_plant == 1), phylum)

@info("Finish spliting the data...")

┌ Info: Finish spliting the data...
└ @ Main In[213]:23


Matrix{Any} (alias for Array{Any, 2})

In [214]:
# Count number of new sample to generate
label_idx = size(phylum)[2]
percentage = [(i, count(==(i), phylum[:,label_idx])) for i in unique(phylum[:,label_idx])]
num_zero = percentage[1][2]
num_one = percentage[2][2]

gen_one = round(0.3 * num_one)
gen_zero = round(num_one*1.3 - num_zero)
@info("Finish calculating number of samples to generate...")

┌ Info: Finish calculating number of samples to generate...
└ @ Main In[214]:9


In [215]:
function generate(data, variety, label)
    new_gen = zeros(0)
    rd = rand(1 : size(data)[1])
    original = data[rd, :]
    append!(new_gen, 114514)
    for i in 2:size(data)[2] - 2
        m = mean(data[:, i])/10
        v = var(data[:, i])/10
        noise = rand(Normal(m,v))
        if original[i] == 0
            append!(new_gen, 0)
        elseif original[i] + noise < 0
            append!(new_gen, 0)
        else
            append!(new_gen, original[i] + noise)
        end
    end
    append!(new_gen, variety)
    append!(new_gen, label)
    return new_gen
end

generate (generic function with 4 methods)

In [296]:

generated = zeros(62, Int(gen_zero+gen_one))
for i in 1:gen_zero
    new_gen = zeros(62)
    var = rand(1:6)
    if var == 1 
        new_gen = generate(red_low,1,0)
    elseif var == 2
        new_gen = generate(white_low,2,0)
    elseif var == 3
        new_gen = generate(yellow_low,3,0)
    elseif var == 4
        new_gen = generate(russet_low,4,0)
    elseif var == 5
        #FIXME
        new_gen = generate(yellow_little_low,5,0)
    else
        new_gen = generate(yellow_little_low,6,0)
    end
    generated[:, Int(i)] = new_gen
end

for i in 1:gen_one
    new_gen = zeros(62)
    var = rand(1:6)
    if var == 1 
        new_gen = generate(red_high,1,1)
    elseif var == 2
        new_gen = generate(white_high,2,1)
    elseif var == 3
        new_gen = generate(yellow_high,3,1)
    elseif var == 4
        new_gen = generate(russet_high,4,1)
    elseif var == 5
        new_gen = generate(yellow_little_high,5,1)
    else
        new_gen = generate(yellow_little_high,6,1)
    end
    generated[:, Int(gen_zero + i)] = new_gen
end
generated = generated'
tmp = generated[:,2:60] ./ 100
generated[:, 2:60] .= tmp
CSV.write("../processed-data/generated_data.csv",  Tables.table(generated), writeheader=false)

"../processed-data/generated_data.csv"